In [1]:
# Note on all the variables used in this code
# PCT_ACS19_5yr_B08134081,% Total:  Subway Or Elevated Rail Light Rail Streetcar Or Trolley

In [2]:
SELECTED_STATE = 'ny'

In [3]:
# import packages
import pandas as pd
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [4]:
# import helper functions
from helpers.variable_helper import derive_variables
from helpers.environment_helper import retrieve_default_state

In [5]:
mny = retrieve_default_state()

In [6]:
mny

'ny'

In [7]:
# Retrieve data and derive appropriate variables
df = pd.read_csv('../data/ny_data/acs_censustract_data/ny_2019_censustract_data.csv', na_values=".", skiprows=[0])

In [8]:
derive_variables(df)

In [9]:
df

,Geo_FIPS,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,...,PCT_SE_A10032_009,PCT_SE_A10032_010,PCT_SE_A10032_011,PCT_SE_A10032_012,SE_A17005_001,SE_A17005_002,SE_A17005_003,PCT_SE_A17005_002,PCT_SE_A17005_003,population_density
0,36001000100,Census Tract 1,"Census Tract 1, Albany County, New York",ny,140,0,ACSSF,3392,NaN,NaN,...,0.53,0.00,0.00,1.7,1015,914,101,90.05,9.95,0.000860
1,36001000200,Census Tract 2,"Census Tract 2, Albany County, New York",ny,140,0,ACSSF,3393,NaN,NaN,...,6.08,5.68,0.00,0.0,2533,2276,257,89.85,10.15,0.002321
2,36001000300,Census Tract 3,"Census Tract 3, Albany County, New York",ny,140,0,ACSSF,3394,NaN,NaN,...,4.86,9.90,0.00,0.0,3131,2693,438,86.01,13.99,0.001067
3,36001000401,Census Tract 4.01,"Census Tract 4.01, Albany County, New York",ny,140,0,ACSSF,3395,NaN,NaN,...,3.55,20.41,0.00,0.0,1043,1017,26,97.51,2.49,0.000261
4,36001000403,Census Tract 4.03,"Census Tract 4.03, Albany County, New York",ny,140,0,ACSSF,3396,NaN,NaN,...,1.40,0.00,0.00,0.0,2797,2641,156,94.42,5.58,0.001355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,36123150100,Census Tract 1501,"Census Tract 1501, Yates County, New York",ny,140,0,ACSSF,8305,NaN,NaN,...,0.95,0.38,14.63,0.0,2931,2780,151,94.85,5.15,0.000034
4914,36123150200,Census Tract 1502,"Census Tract 1502, Yates County, New York",ny,140,0,ACSSF,8306,NaN,NaN,...,0.56,2.28,3.31,0.0,2809,2688,121,95.69,4.31,0.000040
4915,36123150300,Census Tract 1503,"Census Tract 1503, Yates County, New York",ny,140,0,ACSSF,8307,NaN,NaN,...,0.00,0.00,15.73,0.0,2396,2336,60,97.50,2.50,0.000015
4916,36123150400,Census Tract 1504,"Census Tract 1504, Yates County, New York",ny,140,0,ACSSF,8308,NaN,NaN,...,0.00,0.00,2.95,0.0,1826,1727,99,94.58,5.42,0.000053


In [10]:
# remove Geo_FIPS as it's not needed in Lasso Regression
df = df.iloc[: , 1:]

In [11]:
from sklearn.model_selection import train_test_split
scaler = preprocessing.StandardScaler()
y = df['PCT_ACS19_5yr_B08134081']
X = df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081']
# X = df.drop(['pct_walking_and_public_transit'], axis=1, inplace=True)

In [12]:
import numpy as np
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
df=df.dropna()
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
# df = df.replace(np.nan, 0)
# df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [13]:
df.shape

(4918, 1046)

In [14]:
df

,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,Geo_REGION,Geo_DIVISION,...,PCT_SE_A10032_009,PCT_SE_A10032_010,PCT_SE_A10032_011,PCT_SE_A10032_012,SE_A17005_001,SE_A17005_002,SE_A17005_003,PCT_SE_A17005_002,PCT_SE_A17005_003,population_density
0,Census Tract 1,"Census Tract 1, Albany County, New York",ny,140,0,ACSSF,3392,0.0,0.0,0.0,...,0.53,0.00,0.00,1.7,1015,914,101,90.05,9.95,0.000860
1,Census Tract 2,"Census Tract 2, Albany County, New York",ny,140,0,ACSSF,3393,0.0,0.0,0.0,...,6.08,5.68,0.00,0.0,2533,2276,257,89.85,10.15,0.002321
2,Census Tract 3,"Census Tract 3, Albany County, New York",ny,140,0,ACSSF,3394,0.0,0.0,0.0,...,4.86,9.90,0.00,0.0,3131,2693,438,86.01,13.99,0.001067
3,Census Tract 4.01,"Census Tract 4.01, Albany County, New York",ny,140,0,ACSSF,3395,0.0,0.0,0.0,...,3.55,20.41,0.00,0.0,1043,1017,26,97.51,2.49,0.000261
4,Census Tract 4.03,"Census Tract 4.03, Albany County, New York",ny,140,0,ACSSF,3396,0.0,0.0,0.0,...,1.40,0.00,0.00,0.0,2797,2641,156,94.42,5.58,0.001355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,Census Tract 1501,"Census Tract 1501, Yates County, New York",ny,140,0,ACSSF,8305,0.0,0.0,0.0,...,0.95,0.38,14.63,0.0,2931,2780,151,94.85,5.15,0.000034
4914,Census Tract 1502,"Census Tract 1502, Yates County, New York",ny,140,0,ACSSF,8306,0.0,0.0,0.0,...,0.56,2.28,3.31,0.0,2809,2688,121,95.69,4.31,0.000040
4915,Census Tract 1503,"Census Tract 1503, Yates County, New York",ny,140,0,ACSSF,8307,0.0,0.0,0.0,...,0.00,0.00,15.73,0.0,2396,2336,60,97.50,2.50,0.000015
4916,Census Tract 1504,"Census Tract 1504, Yates County, New York",ny,140,0,ACSSF,8308,0.0,0.0,0.0,...,0.00,0.00,2.95,0.0,1826,1727,99,94.58,5.42,0.000053


In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV

# https://towardsdatascience.com/feature-selection-in-machine-learning-using-lasso-regression-7809c7c2771a


In [16]:
# pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,10,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )
search.fit(X_train,y_train)


Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END ....................model__alpha=0.1;, score=nan total time=   0.1s
[CV 2/5] END ....................model__alpha=0.1;, score=nan total time=   0.1s
[CV 3/5] END ....................model__alpha=0.1;, score=nan total time=   0.1s
[CV 4/5] END ....................model__alpha=0.1;, score=nan total time=   0.1s
[CV 5/5] END ....................model__alpha=0.1;, score=nan total time=   0.1s
[CV 1/5] END ....................model__alpha=0.2;, score=nan total time=   0.1s
[CV 2/5] END ....................model__alpha=0.2;, score=nan total time=   0.1s
[CV 3/5] END ....................model__alpha=0.2;, score=nan total time=   0.1s
[CV 4/5] END ....................model__alpha=0.2;, score=nan total time=   0.1s
[CV 5/5] END ....................model__alpha=0.2;, score=nan total time=   0.1s
[CV 1/5] END ....model__alpha=0.30000000000000004;, score=nan total time=   0.0s
[CV 2/5] END ....model__alpha=0.300000000000000

c:\Users\cjong\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
495 fits failed out of a total of 495.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
99 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\cjong\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\cjong\anaconda3\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\cjong\anaconda3\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\cjong\anaco

ValueError: could not convert string to float: 'Census Tract 4168.01'

In [ ]:
search.best_params_

{'model__alpha': 0.1}

In [ ]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [ ]:
importance = np.abs(coefficients)

In [ ]:
result = np.array(df.loc[:, df.columns != 'PCT_ACS19_5yr_B08134081'].columns)[importance > 0]

In [ ]:
df_columns = pd.read_csv('./raw_data/2019_data_columns.csv', na_values=".")
df_columns

,name,decode
0,Geo_FIPS,FIPS
1,Geo_NAME,Area Name
2,Geo_QName,Qualifying Name
3,Geo_STUSAB,State Postal Abbreviation
4,Geo_SUMLEV,Summary Level
...,...,...
988,ACS19_5yr_B19013A001s,Std. Error: Median Household Income In The Pas...
989,ACS19_5yr_B19013B001,Median Household Income In The Past 12 Months ...
990,ACS19_5yr_B19013B001s,Std. Error: Median Household Income In The Pas...
991,ACS19_5yr_B19013D001,Median Household Income In The Past 12 Months ...


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_columns.loc[df_columns['name'].isin(result)]

,name,decode
460,PCT_ACS19_5yr_B08134011,% Total: Car Truck Or Van
510,PCT_ACS19_5yr_B08134061,% Total: Public Transportation (Excluding Taxicab)
531,PCT_ACS19_5yr_B08134082,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): Less Than 10 Minutes
532,PCT_ACS19_5yr_B08134083,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 10 To 14 Minutes
533,PCT_ACS19_5yr_B08134084,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 15 To 19 Minutes
534,PCT_ACS19_5yr_B08134085,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 20 To 24 Minutes
535,PCT_ACS19_5yr_B08134086,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 25 To 29 Minutes
536,PCT_ACS19_5yr_B08134087,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 30 To 34 Minutes
537,PCT_ACS19_5yr_B08134088,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 35 To 44 Minutes
538,PCT_ACS19_5yr_B08134089,% Total: Subway Or Elevated Rail Light Rail Streetcar Or Trolley (Carro PÃºblico In Puerto Rico): 45 To 59 Minutes


In [ ]:
df_columns.loc[df_columns['name'].isin(result)].iloc[:,0].values.tolist()

['PCT_ACS19_5yr_B08134011',
 'PCT_ACS19_5yr_B08134061',
 'PCT_ACS19_5yr_B08134082',
 'PCT_ACS19_5yr_B08134083',
 'PCT_ACS19_5yr_B08134084',
 'PCT_ACS19_5yr_B08134085',
 'PCT_ACS19_5yr_B08134086',
 'PCT_ACS19_5yr_B08134087',
 'PCT_ACS19_5yr_B08134088',
 'PCT_ACS19_5yr_B08134089',
 'PCT_ACS19_5yr_B08134090',
 'PCT_ACS19_5yr_B08134091',
 'ACS19_5yr_B08301021',
 'PCT_ACS19_5yr_B08301012',
 'PCT_ACS19_5yr_B08301014',
 'PCT_ACS19_5yr_B08301021']